In [ ]:
if __name__ == "__main__":
    # set args
    api_fp = '../../admin/cmc.txt'
    start_date = date(2015, 1, 1)
    end_date   = date(2023, 2, 1)
    base_url = "https://pro-api.coinmarketcap.com"

    # import api key
    with open(api_fp) as f:
        API_KEY = f.readlines()
        API_KEY = API_KEY[0].strip()
    
    # confirm api is working
    session = initiateAPI(base_url)

    # map cmc universe to coinapi

    # manually go through tokens that are not in coinapi to ensure they useless

    # grab all spot markets for each of the tokens on the us exchanges

    # for each market, convert price to usd and take average as the token price; sum the volume

    

### (0) Cancel API!

In [394]:
# CoinAPI key

fp = '../0-admin/coinapi.txt'

with open(fp) as f:
    API_KEY = f.readlines()
    API_KEY = API_KEY[0].strip()


In [399]:
import requests
url = 'https://www.coinapi.io/api/subscriptions/cancel'
headers = {'X-CoinAPI-Key' : API_KEY}
response = requests.get(url, headers=headers)


In [401]:
response.status_code

400

In [402]:
API_KEY

'E4D2951E-3901-4AFB-AC0E-495094975A73'

### (0) Preliminaries

In [192]:
#!pip install --upgrade coinapi_rest_v1

In [80]:
import requests

url = "https://api.exchange.coinbase.com/products/btc/candles

headers = {"Accept": "application/json"}

response = requests.get(url, headers=headers)

print(response.text)

,Open,High,Low,Close,Volume
Date,,,,,
2022-08-11 11:44:00,24580.52,24599.17,24591.57,24598.77,9.981655
2022-08-11 11:45:00,24590.86,24613.37,24596.54,24609.37,13.030282
2022-08-11 11:46:00,24606.92,24639.77,24610.83,24626.42,30.776527
2022-08-11 11:47:00,24618.83,24632.98,24625.54,24628.08,8.644579
2022-08-11 11:48:00,24623.36,24640.73,24629.11,24629.90,11.868355
...,...,...,...,...,...
2022-08-11 16:39:00,24297.86,24316.61,24316.61,24301.54,13.787485
2022-08-11 16:40:00,24297.58,24320.27,24300.72,24320.27,22.383356
2022-08-11 16:41:00,24297.41,24322.57,24321.63,24317.08,18.586477


In [193]:
from coinapi_rest_v1.restapi import CoinAPIv1
import datetime, sys
import pandas as pd
import numpy as np
import re


In [194]:
# CoinAPI key

fp = '../0-admin/coinapi.txt'

with open(fp) as f:
    API_KEY = f.readlines()
    API_KEY = API_KEY[0].strip()


In [196]:
# Initiate CoinAPI
api = CoinAPIv1(API_KEY)


### (1) Determine the investable token universe

In [218]:
# Obtain all coins on CoinAPI to be filtered down
symbols = api.metadata_list_symbols()


In [300]:
# How many do I lose if I drop Huobi because i cant trade there?
# Lost 215 mostly irrelevant tokens from dropping binance
# Lost 113 mostly irrelevant tokens from dropping FTX
# Lost 113 tokens from dropping HUOBI with a few big time ones
#   (e.g. billiosn MCAP) and a couple dozen small players, e.g. $10MM's mcap
# Lost 172 tokens from dropping OKEX with dozens of legit players 
#   in 10s/100s MM and other irrelevant
# Lost 136 tokens from dropping BITFINEX with a dozen big boys in billions; 
#   and dozens in tens/100s MMs and rest irrelevant


In [301]:
# Find the coins available on major US exchanges
relevant_exchanges = ['BINANCEUS', 'FTXUS', 'COINBASE',
                      'KUCOIN', 'KRAKEN', 'GEMINI']
rel_symbols = []
for symbol in symbols:
    if (symbol['exchange_id'] in relevant_exchanges):
        if (symbol['symbol_type'] in ['SPOT']):
            rel_symbols.append(symbol)

In [302]:
# Find all unique tokens
temp_list = []
for symbol in rel_symbols:
    temp_list.append(symbol['asset_id_base'])
unique_tokens = list(np.unique(np.array(temp_list)))


In [303]:
# Remove tokens that are derivatives of other tokens
token_names_to_remove = ['DOWN', 'BEAR', 'BULL', 'HALF', 'HEDGE', 'UP', '3L', '3S', 'USD']
for string in token_names_to_remove:
    unique_tokens = [i for i in unique_tokens if not (string in i)]


In [304]:
# Remove stable coins    
unique_tokens = list(unique_tokens)
currencies_to_remove = ['EUR', 'AUD', 'GBP', 'CAD', 
                        'KRW', 'MTL', 'AOA', 'ERN']
for c in currencies_to_remove:
    unique_tokens.remove(c)


In [305]:
# Put unique tokens into data frame
coinapi_df = pd.DataFrame(data = unique_tokens)
coinapi_df = coinapi_df.rename(columns = {0: 'symbol'})


In [306]:
# Read in coins listed on Messari from CSV downloaded
messari_df = pd.read_csv('../3-data/raw/Messari-Screener-Dec-02-2021-1638465942.csv')

# Drop rows from Messari list that are duplicate coins and not available in US
coins_not_avail_in_us_and_dup = ['MIR COIN', 'Carry', 'SuperCoin', 'Monavale',
                                 'Hydro Protocol', 'Name Changing Token',
                                 'Memetic / PepeCoin', 'CryptoTask', 'Penta',
                                 'Comet', 'LINA', 'PlayChip', 'XRIBA',
                                 'World Bitcoin', 'StarkWare', 'Tari',
                                 'PieDAO Metaverse NFT Index', 'Truebit',
                                 'DeGate', 'Decent', 'Oxygen', 'Clover Finance',
                                 'Somnium Space', 'XTOKEN', 'Unique One', 
                                 'Selfkey']
messari_df = messari_df[~messari_df.name.isin(coins_not_avail_in_us_and_dup)]

# Drop rows that are stablecoins
stablecoins_to_drop = ['Ultra Salescloud']
messari_df = messari_df[~messari_df.name.isin(stablecoins_to_drop)]
messari_df = messari_df[messari_df.sector != 'Stablecoins']



In [307]:
# Merge together messari coins with coin api tokens
token_df = messari_df.merge(coinapi_df,
                            how='outer',
                            on='symbol',
                            validate='one_to_one',
                            indicator=True)

In [308]:
# Remove tokens manually
symbols_dont_map_to_token = ['ARN', 'AXP', 'BARY', 'BCHA', 'BCHSV', 'BKCAT', 
                             'BTMX', 'CFD', 'CGLD', 'DADI', 'ENDOR',
                             'ETH2', 'ETH2.S', 'GALAX', 'HAT', 'HAV',
                             'HKN', 'HMCN', 'HOTNOW', 'HOTT', 'HSR',
                             'ICN', 'INS', 'OPQ', 'MCO2', 'MODEFI', 
                             'PAX', 'PRL', 'REPV2',
                             'RHOC', 'RPX', 'SHL', 'SNTR', 'TIO', 'VEN', 'VRAB',
                             'XRB', 'YFDAI', 'YOP', 'ZEL']
symbols_with_token_below_mcap_1m_ever = ['ACOIN', 'BOS', 'CEUR', 'COFI', 'DACC', 'DCC',
                                         'FORESTPLUS', 'GLA', 'J8T', 'LNCHX', 'MUSH', 'ONOT',
                                         'PARETO', 'PAZZI', 'PBL', 'PMGT', 'PURA', 'SNOV',
                                         'SPF', 'STQ', 'TRAXIA', 'UKG', 'VNX', 'VSX', 
                                         'WAX', 'WGP', 'XLR', 'ZINC']
stablecoin_symbols_to_drop = ['CUSD', 'MUSD', 'NUSD', 'DAI']
symbols_without_market_to_usd_or_stablecoin = ['BLZ', 'COVAL', 'DESO', 'IDEX', 'KLAY', 
                                               'LINA', 'LOKI', 'MDT', 'RBN',
                                               'SMT', 'SPELL', 'TRVL', 'TRTL', 'UNB', 'XEC']
wrapped_tokens_to_drop = ['EBTC', 'EFIL', 'TBTC', 'RBTC', 'WBTC']
symbols_only_recently_started_trading = ['FOX', 'GGG']
token_df = token_df[~(token_df.symbol.isin(symbols_dont_map_to_token))]
token_df = token_df[~(token_df.symbol.isin(symbols_with_token_below_mcap_1m_ever))]
token_df = token_df[~(token_df.symbol.isin(stablecoin_symbols_to_drop))]
token_df = token_df[~(token_df.symbol.isin(wrapped_tokens_to_drop))]
token_df = token_df[~(token_df.symbol.isin(symbols_without_market_to_usd_or_stablecoin))]
token_df = token_df[~(token_df.symbol.isin(symbols_only_recently_started_trading))]

In [309]:
# DROP COINS IN MESSARI BUT NOT PURCHASABLE
token_df = token_df[token_df._merge != 'left_only']
token_df = token_df.drop('_merge', axis=1)


### (2) Pull historical data for each token

In [310]:
# Build list of tokens to pull historical prices for

tokens = token_df.symbol.values
assert(len(np.unique(tokens)) == len(tokens))
tokens = list(np.unique(tokens)) # sort alphabetically

In [311]:
# Create custom function to subset down all markets
#     for a token to relevant ones per my logic of
#     on exchange I can access and liquid pair, e.g.
#     USD, USDT, or BTC
def findRelevantMarket(token, all_markets, pair, exchanges_in_order_by_historical_length):
    rel_market = ''

    for exchange in exchanges_in_order_by_historical_length:
        temp = [m for m in all_markets[exchange] if token + pair in m]
        if pair == '_USD':
            temp = [m for m in all_markets[exchange] if (token + pair in m) and ('USDT' not in m)]
            temp = [m for m in all_markets[exchange] if (token + pair in m) and ('USDC' not in m)]
        if temp:
            rel_market = temp
            break

    return rel_market

In [312]:
# Figure the relevant market for all tokens
exchanges_in_order_by_historical_length = ['KRAKEN', 'KUCOIN', 'COINBASE',   
                                           'BINANCEUS', 'FTXUS', 'GEMINI']
markets = []
for i in range(0,len(tokens)):
    token = tokens[i]

    # Determine all the relevant markets for the token
    all_markets = dict.fromkeys(relevant_exchanges, [])
    for symbol in symbols:
        if (symbol['exchange_id'] in relevant_exchanges):
            try:
                if symbol['asset_id_base'] == token:
                    all_markets[symbol['exchange_id']] = all_markets[symbol['exchange_id']] + [symbol['symbol_id']]
            except KeyError:
                pass
    
    # Determine USD market using first exchange with one by volume
    market = ''
    market = findRelevantMarket(token, all_markets, '_BTC', exchanges_in_order_by_historical_length)

    # If no USD market, let's look for BTC, then USDT, then ETH
    if not market:
        market = findRelevantMarket(token, all_markets, '_USDT', exchanges_in_order_by_historical_length)
    if not market:
        market = findRelevantMarket(token, all_markets, '_ETH', exchanges_in_order_by_historical_length)
    if not market:
        market = findRelevantMarket(token, all_markets, '_USD', exchanges_in_order_by_historical_length)
    if not market:
        market = findRelevantMarket(token, all_markets, '_USDC', exchanges_in_order_by_historical_length)
    if not market:
        print("Didn't find a market for " + token)
        break

    assert(type(market) is list)
    market = market[0]
    assert(type(market) is str)
    assert(len(market) > 0) # Ensure we at least found some market
    markets.append(market)
    

In [313]:
# Manually fix problematic markets
token = 'OMG'
index = tokens.index(token)
markets[index] = 'KRAKEN_SPOT_OMG_USD'

token = 'SERO'
index = tokens.index(token)
markets[index] = 'KUCOIN_SPOT_SERO_USDT'

token = 'GYEN'
index = tokens.index(token)
markets[index] = 'COINBASE_SPOT_GYEN_USD'

token = 'BTC'
index = tokens.index(token)
markets[index] = 'COINBASE_SPOT_BTC_USD'

token = 'ETH'
index = tokens.index(token)
markets[index] = 'KRAKEN_SPOT_ETH_USD'

token = 'YFII'
index = tokens.index(token)
markets[index] = 'COINBASE_SPOT_YFII_USD'

token = 'HNT'
index = tokens.index(token)
markets[index] = 'BINANCEUS_SPOT_HNT_USD'


In [20]:
# Pull the historical data for all tokens
# NOTE: BIG API CALL!!!
ohlcvs = []
start_of_2009 = datetime.date(2009, 1, 1).isoformat()
today = datetime.date(2022, 1, 4).isoformat()
for i in range(0,len(markets)):
    if (i % 20) == 0:
        print(i)
    market = markets[i]
    print(market)
    print('\n')
    ohlcv = api.ohlcv_historical_data(market, {'period_id': '1DAY', 
                                               'time_start': start_of_2009,
                                               'time_end': today,
                                               'limit': 5000})

    ohlcvs.append(ohlcv)

0
COINBASE_SPOT_1INCH_BTC


KUCOIN_SPOT_2CRZ_BTC


KRAKEN_SPOT_AAVE_BTC


KUCOIN_SPOT_ABBC_BTC


KUCOIN_SPOT_ABT_BTC


KUCOIN_SPOT_ACAT_BTC


KUCOIN_SPOT_ACE_USDT


COINBASE_SPOT_ACH_USD


KUCOIN_SPOT_ACT_BTC


KRAKEN_SPOT_ADA_BTC


KUCOIN_SPOT_ADB_BTC


KUCOIN_SPOT_ADS_BTC


KUCOIN_SPOT_ADX_USDT


KUCOIN_SPOT_AERGO_BTC


KUCOIN_SPOT_AGI_BTC


KUCOIN_SPOT_AGIX_BTC


KUCOIN_SPOT_AGLD_USDT


KUCOIN_SPOT_AI_USDT


KUCOIN_SPOT_AION_BTC


KUCOIN_SPOT_AIOZ_USDT


20
KUCOIN_SPOT_AIX_BTC


KUCOIN_SPOT_AKRO_BTC


KUCOIN_SPOT_ALBT_USDT


COINBASE_SPOT_ALCX_USDT


KUCOIN_SPOT_ALEPH_USDT


KRAKEN_SPOT_ALGO_BTC


KUCOIN_SPOT_ALICE_BTC


KUCOIN_SPOT_ALPACA_USDT


KUCOIN_SPOT_ALPHA_BTC


KUCOIN_SPOT_AMB_BTC


KUCOIN_SPOT_AMIO_BTC


COINBASE_SPOT_AMP_USD


KUCOIN_SPOT_AMPL_BTC


KUCOIN_SPOT_ANC_USDT


KRAKEN_SPOT_ANKR_BTC


KRAKEN_SPOT_ANT_BTC


KUCOIN_SPOT_APH_BTC


KUCOIN_SPOT_API3_USDT


KUCOIN_SPOT_APL_BTC


KUCOIN_SPOT_AR_BTC


40
KUCOIN_SPOT_ARPA_USDT


KUCOIN_SPOT_ARRR_BTC


KUCOIN_SPOT_ARX_USD

KUCOIN_SPOT_MATTER_USDT


KUCOIN_SPOT_MEET_BTC


KUCOIN_SPOT_MEM_USDT


KUCOIN_SPOT_META_BTC


KUCOIN_SPOT_MFT_BTC


KUCOIN_SPOT_MHC_BTC


360
KRAKEN_SPOT_MINA_BTC


COINBASE_SPOT_MIR_BTC


KUCOIN_SPOT_MITX_BTC


KRAKEN_SPOT_MKR_BTC


KUCOIN_SPOT_MLK_BTC


KRAKEN_SPOT_MLN_BTC


KUCOIN_SPOT_MNST_USDT


KUCOIN_SPOT_MNW_USDT


KUCOIN_SPOT_MOBI_BTC


KUCOIN_SPOT_MOD_BTC


KUCOIN_SPOT_MONI_USDT


KUCOIN_SPOT_MOVR_USDT


KUCOIN_SPOT_MSWAP_BTC


KUCOIN_SPOT_MTC_BTC


KUCOIN_SPOT_MTH_BTC


KUCOIN_SPOT_MTN_BTC


KUCOIN_SPOT_MTRG_USDT


KUCOIN_SPOT_MTV_BTC


KUCOIN_SPOT_MVP_BTC


KUCOIN_SPOT_MWAT_BTC


380
KUCOIN_SPOT_MXC_USDT


KUCOIN_SPOT_MXW_USDT


KUCOIN_SPOT_NAKA_USDT


KRAKEN_SPOT_NANO_BTC


KUCOIN_SPOT_NDAU_USDT


KUCOIN_SPOT_NEAR_BTC


KUCOIN_SPOT_NEBL_BTC


KUCOIN_SPOT_NEO_BTC


KUCOIN_SPOT_NFT_USDT


KUCOIN_SPOT_NFTB_USDT


KUCOIN_SPOT_NGC_USDT


KUCOIN_SPOT_NGM_USDT


KUCOIN_SPOT_NIF_USDT


KUCOIN_SPOT_NIM_BTC


KUCOIN_SPOT_NIX_BTC


KUCOIN_SPOT_NKN_BTC


KRAKEN_SPOT_NMC_USD


KUCOIN_

In [21]:
# Create a function to do an initial clean of the ohlcv data
def cleanOHLCV(ohlcv, market, token, btc_df=None, eth_df=None, 
               usdt_df=None, usdc_df=None):
    token = token.lower()
    df = pd.DataFrame(ohlcv)
    df = df[['time_close', 'price_close', 'volume_traded']]
    
    df['time_close'] = df.time_close.apply(lambda x: pd.to_datetime(x).strftime('%Y-%m-%d'))
    df = df.rename(columns = {'time_close': 'date'})
    
    assert(type(df.price_close.values[0]) == np.float64)
    assert(type(df.volume_traded.values[0]) == np.float64)
    assert(sum(df.price_close >= 0) == df.shape[0])
    assert(sum(df.volume_traded >= 0) == df.shape[0])
    
    # Figure out the quote token
    index = [i.start() for i in re.finditer('_', market)][-1]
    quote_token = market[index+1:]

    # Convert price and volume appropriately into USD
    if quote_token == 'BTC':
        df = df.merge(btc_df,
                      how='inner',
                      on='date',
                      validate='one_to_one')
        df = df.drop(columns = ['btc_usd_volume'])
        df[token + '_usd'] = df.price_close * df.btc_usd
        df[token + '_usd_volume'] = df.volume_traded * df.price_close * df.btc_usd
        df = df.drop(columns = ['btc_usd'])
    elif quote_token == 'USDT':
        df = df.merge(usdt_df,
                      how='inner',
                      on='date',
                      validate='one_to_one')
        df[token + '_usd'] = df.price_close * df.usdt_usd
        df[token + '_usd_volume'] = df.volume_traded * df.price_close * df.usdt_usd
        df = df.drop(columns = ['usdt_usd'])
    elif quote_token == 'USD':
        df[token + '_usd'] = df.price_close
        df[token + '_usd_volume'] = df.volume_traded * df.price_close
    elif quote_token == 'ETH':
        df = df.merge(eth_df,
                      how='inner',
                      on='date',
                      validate='one_to_one')
        df = df.drop(columns = ['eth_usd_volume'])
        df[token + '_usd'] = df.price_close * df.eth_usd
        df[token + '_usd_volume'] = df.volume_traded * df.price_close * df.eth_usd
        df = df.drop(columns = ['eth_usd'])
    elif quote_token == 'USDC':
        df = df.merge(usdc_df,
                      how='inner',
                      on='date',
                      validate='one_to_one')
        df[token + '_usd'] = df.price_close * df.usdc_usd
        df[token + '_usd_volume'] = df.volume_traded * df.price_close * df.usdc_usd
        df = df.drop(columns = ['usdc_usd'])
    else:
        print('Problem quote is not in valid set!')

    # Drop unnecssary columns
    df = df.drop(columns = ['price_close', 'volume_traded'])
    
    return df

In [22]:
# Obtain USDT price to use for exchange rate conversions
usdt_df = pd.read_csv('../3-data/raw/historical_data_tether_from_messari.csv')
usdt_df['date'] = pd.to_datetime(usdt_df.Date).dt.strftime('%Y-%m-%d')
usdt_df['usdt_usd'] = usdt_df['Price (Close)']
usdt_df = usdt_df.drop(columns = ['Date', 'Price (Close)'])
usdt_df = usdt_df.sort_values('date').reset_index(drop=True)

In [23]:
# Obtain USDC price to use for exchange rate conversions
usdc_df = pd.read_csv('../3-data/raw/historical_data_usdc_from_messari.csv')
usdc_df['date'] = pd.to_datetime(usdc_df.Date).dt.strftime('%Y-%m-%d')
usdc_df['usdc_usd'] = usdc_df['Price (Close)']
usdc_df = usdc_df.drop(columns = ['Date', 'Price (Close)'])
usdc_df = usdc_df.sort_values('date').reset_index(drop=True)

In [24]:
# Build the price time series for BTC and ETH
#     given I will have to use these to conver the
#     price of others
btc_index  = tokens.index('BTC')
eth_index  = tokens.index('ETH')

btc_ohlcv = ohlcvs[btc_index]
eth_ohlcv = ohlcvs[eth_index]

btc_df  = cleanOHLCV(btc_ohlcv, markets[btc_index], tokens[btc_index])
eth_df  = cleanOHLCV(eth_ohlcv, markets[eth_index], tokens[eth_index])

In [72]:
# Convert ohlcv objects to data frames
dfs = []
for i in range(0, len(ohlcvs)):
    ohlcv = ohlcvs[i]
    market = markets[i]
    token = tokens[i]
    print(token)
    df = cleanOHLCV(ohlcv, market, token, 
                    btc_df=btc_df, eth_df=eth_df, usdt_df=usdt_df, usdc_df=usdc_df)
    dfs.append(df)

1INCH
2CRZ
AAVE
ABBC
ABT
ACAT
ACE
ACH
ACT
ADA
ADB
ADS
ADX
AERGO
AGI
AGIX
AGLD
AI
AION
AIOZ
AIX
AKRO
ALBT
ALCX
ALEPH
ALGO
ALICE
ALPACA
ALPHA
AMB
AMIO
AMP
AMPL
ANC
ANKR
ANT
APH
API3
APL
AR
ARPA
ARRR
ARX
ASD
ASH
ASM
ATA
ATOM
AUCTION
AUDIO
AURY
AVA
AVAX
AXC
AXPR
AXS
BADGER
BAKE
BAL
BAND
BASIC
BAT
BAX
BCD
BCH
BCHA
BCHABC
BCPT
BEPRO
BHC
BLOC
BLOK
BMON
BNB
BNC
BNS
BNT
BNTY
BOA
BOLT
BOND
BONDLY
BOSON
BPT
BRD
BRG
BRZ
BTC
BTG
BTM
BTRST
BTT
BU
BURGER
BURP
BUX
BUY
BZRX
C98
CAG
CAKE
CAN
CAPP
CARD
CARR
CAS
CBC
CELO
CERE
CFG
CGG
CHP
CHR
CHSB
CHZ
CIRUS
CIX100
CKB
CLV
COMB
COMP
COSM
COTI
COV
CPC
CPOOL
CQT
CREAM
CRO
CRPT
CRV
CS
CSP
CTI
CTR
CTSI
CTX
CUBE
CUDOS
CV
CVC
CWAR
CWS
CXO
DAG
DAO
DAPPT
DAPPX
DASH
DAT
DATA
DATX
DBC
DCR
DDX
DEB
DEGO
DENT
DERO
DEXE
DFI
DFYN
DGB
DGTX
DIA
DINO
DIVI
DMG
DMTR
DNA
DNT
DOCK
DODO
DOGE
DORA
DOT
DPET
DPI
DPR
DREAMS
DRGN
DSLA
DTA
DVPN
DX
DYDX
DYP
EDG
EDGE
EDN
EFX
EGLD
EGT
ELA
ELEC
ELF
ELIX
ELON
ENJ
ENQ
ENS
EOS
EOSC
EPIK
EPS
EQX
EQZ
ERG
ERSDL
ETC
ETH
ETHO
ETN
EVX
EWT
EXRD
EXY


In [73]:
# Merge together all data frames to save
df = dfs[0]
for i in range(1,len(dfs)):
    df = df.merge(dfs[i],
                  how='outer',
                  on='date',
                  validate='one_to_one')
    
df = df.sort_values('date')
df = df.reset_index(drop=True)

### (3) Obtain mcaps from coinmarketcap

In [120]:
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json

In [121]:
# Set up coinmarket cap API key
fp = '../0-admin/coinmarketcap.txt'

with open(fp) as f:
    API_KEY = f.readlines()
    API_KEY = API_KEY[0].strip()
    

In [122]:
# Replace token names with coinmarketcap version
cmc_tokens = tokens.copy()
cmc_tokens.pop(cmc_tokens.index('AGI'))
cmc_tokens[cmc_tokens.index('IOTA')] = 'MIOTA'
cmc_tokens[cmc_tokens.index('NANO')] = 'XNO'
cmc_tokens[cmc_tokens.index('R')] = 'REV'

In [123]:
# Pull in coinmarketcap data to get the mcaps
url = f'https://pro-api.coinmarketcap.com/v1/cryptocurrency/quotes/latest'

print(url)
headers = {
    'Accepts': 'application/json',
    'X-CMC_PRO_API_KEY': API_KEY,
}
parameters = {
'symbol':",".join(cmc_tokens)

}

session = Session()
session.headers.update(headers)
try:
    response = session.get(url, params=parameters)
    data = json.loads(response.text)
except (ConnectionError, Timeout, TooManyRedirects) as e:
    data = json.loads(response.text)
    

https://pro-api.coinmarketcap.com/v1/cryptocurrency/quotes/latest


In [124]:
# Process the CMC data
cmc_matched_tokens = []
cmc_mcaps          = []
for t in data['data']:
    if data['data'][t]['max_supply'] != None:
        cmc_matched_tokens.append(t.upper())
        cmc_mcaps.append(data['data'][t]['max_supply'])
    elif data['data'][t]['total_supply'] != None:
        cmc_matched_tokens.append(t.upper())
        cmc_mcaps.append(data['data'][t]['total_supply'])
    else:
        print('Did not find an mcap!')
        print(t)
        print(data['data'][t])
        print('\n')

assert(len(cmc_matched_tokens) == len(cmc_mcaps))

cmc_df = pd.DataFrame(data = {'token': cmc_matched_tokens, 'mcap': cmc_mcaps})

Did not find an mcap!
APH
{'id': 220, 'name': 'Aphroditecoin', 'symbol': 'APH', 'slug': 'aphroditecoin', 'num_market_pairs': None, 'date_added': '2014-03-22T00:00:00.000Z', 'tags': ['mineable'], 'max_supply': None, 'circulating_supply': None, 'total_supply': None, 'is_active': 0, 'platform': None, 'cmc_rank': None, 'is_fiat': 0, 'last_updated': '2022-01-05T19:00:40.053Z', 'quote': {'USD': {'price': 0, 'volume_24h': None, 'volume_change_24h': None, 'percent_change_1h': None, 'percent_change_24h': None, 'percent_change_7d': None, 'percent_change_30d': None, 'percent_change_60d': None, 'percent_change_90d': None, 'market_cap': None, 'market_cap_dominance': None, 'fully_diluted_market_cap': None, 'last_updated': '2022-01-05T19:00:40.053Z'}}}




In [125]:
# Change CMC token names back to those that match my token list
cmc_df.loc[cmc_df.token == 'MIOTA', 'token'] = 'IOTA'
cmc_df.loc[cmc_df.token == 'XNO', 'token'] = 'NANO'
cmc_df.loc[cmc_df.token == 'REV', 'token'] = 'R'

new_row_df = pd.DataFrame(data = {'token': 'AGI', 'mcap': cmc_df[cmc_df.token == 'AGIX'].mcap})
cmc_df = cmc_df.append(new_row_df)
cmc_df = cmc_df.reset_index(drop=True)

In [126]:
# Drop APX from my side as I am not able to find it in the coinmarketcap data
#df = df.drop(columns = ['aph_usd', 'aph_usd_volume'])
aph_index = tokens.index('APH')
tokens.pop(aph_index)
markets.pop(aph_index)
ohlcvs.pop(aph_index)

[{'time_period_start': '2020-12-31T00:00:00.0000000Z',
  'time_period_end': '2021-01-01T00:00:00.0000000Z',
  'time_open': '2020-12-31T09:30:44.2070000Z',
  'time_close': '2020-12-31T23:59:49.1230000Z',
  'price_open': 1.777,
  'price_high': 1.888,
  'price_low': 1.593,
  'price_close': 1.654,
  'volume_traded': 102798.65972417,
  'trades_count': 4704},
 {'time_period_start': '2021-01-01T00:00:00.0000000Z',
  'time_period_end': '2021-01-02T00:00:00.0000000Z',
  'time_open': '2021-01-01T00:00:16.2230000Z',
  'time_close': '2021-01-01T23:59:35.4070000Z',
  'price_open': 1.654,
  'price_high': 2.204,
  'price_low': 1.608,
  'price_close': 1.81,
  'volume_traded': 143093.32987137,
  'trades_count': 8345},
 {'time_period_start': '2021-01-02T00:00:00.0000000Z',
  'time_period_end': '2021-01-03T00:00:00.0000000Z',
  'time_open': '2021-01-02T00:00:04.5080000Z',
  'time_close': '2021-01-02T23:54:10.4800000Z',
  'price_open': 1.81,
  'price_high': 2.059,
  'price_low': 1.71,
  'price_close': 1.9

### (4) Merge together price+volume data and mcap data

In [127]:
# Obtain the list of tokens to create columns for
for token in tokens:
    print(token)
    mcap = cmc_df[cmc_df.token == token.upper()].mcap.values[0]
    assert(mcap >= 0)
    df[token.lower() + '_usd_mcap'] = df[token.lower() + '_usd'] * mcap


1INCH
2CRZ
AAVE
ABBC
ABT
ACAT
ACE
ACH
ACT
ADA
ADB
ADS
ADX
AERGO
AGI
AGIX
AGLD
AI
AION
AIOZ
AIX
AKRO
ALBT
ALCX
ALEPH
ALGO
ALICE
ALPACA
ALPHA
AMB
AMIO
AMP
AMPL
ANC
ANKR
ANT
API3
APL
AR
ARPA
ARRR
ARX
ASD
ASH
ASM
ATA
ATOM
AUCTION
AUDIO
AURY
AVA
AVAX
AXC
AXPR
AXS
BADGER
BAKE
BAL
BAND
BASIC
BAT
BAX
BCD
BCH
BCHABC
BCPT
BEPRO
BHC
BLOC
BLOK
BMON
BNB
BNC
BNS
BNT
BNTY
BOA
BOLT
BOND
BONDLY
BOSON
BPT
BRD
BRG
BRZ
BTC
BTG
BTM
BTRST
BTT
BU
BURGER
BURP
BUX
BUY
BZRX
C98
CAG
CAKE
CAN
CAPP
CARD
CARR
CAS
CBC
CELO
CERE
CFG
CGG
CHP
CHR
CHSB
CHZ
CIRUS
CIX100
CKB
CLV
COMB
COMP
COSM
COTI
COV
CPC
CPOOL
CQT
CREAM
CRO
CRPT
CRV
CS
CSP
CTI
CTR
CTSI
CTX
CUBE
CUDOS
CV
CVC
CWAR
CWS
CXO
DAG
DAO
DAPPT
DAPPX
DASH
DAT
DATA
DATX
DBC
DCR
DDX
DEB
DEGO
DENT
DERO
DEXE
DFI
DFYN
DGB
DGTX
DIA
DINO
DIVI
DMG
DMTR
DNA
DNT
DOCK
DODO
DOGE
DORA
DOT
DPET
DPI
DPR
DREAMS
DRGN
DSLA
DTA
DVPN
DX
DYDX
DYP
EDG
EDGE
EDN
EFX
EGLD
EGT
ELA
ELEC
ELF
ELIX
ELON
ENJ
ENQ
ENS
EOS
EOSC
EPIK
EPS
EQX
EQZ
ERG
ERSDL
ETC
ETH
ETHO
ETN
EVX
EWT
EXRD
EXY
FARM
FCL


In [128]:
# Sort columns in appropriate order
df = df[['date'] + list(np.sort(df.columns.values[1:]))]

In [129]:
# Sort columns in appropriate order
df = df[['date'] + list(np.sort(df.columns.values[1:]))]

# Convert date column to datetime object and set it as the index
df.index = pd.to_datetime(df.date)
df = df.drop(columns = 'date')

In [130]:
# Manually edit data for some tokens to drop early prices
df.loc[(df.index.year == 2019) & 
           (df.index.month.isin([1,2])), 'bchabc_usd'] = np.nan
df.loc[(df.index.year == 2019) & 
           (df.index.month.isin([1,2])), 'bchabc_usd_mcap'] = np.nan
df.loc[(df.index.year == 2019) & 
           (df.index.month.isin([1,2])), 'bchabc_usd_volume'] = np.nan

df.loc[df.index.year == 2018, 'open_usd'] = np.nan
df.loc[df.index.year == 2018, 'open_usd_mcap'] = np.nan
df.loc[df.index.year == 2018, 'open_usd_volume'] = np.nan

df.loc[df.index.year == 2018, 'stx_usd'] = np.nan
df.loc[df.index.year == 2018, 'stx_usd_mcap'] = np.nan
df.loc[df.index.year == 2018, 'stx_usd_volume'] = np.nan

df.loc[df.index == datetime.date(2020, 9, 30).isoformat(), 'trac_usd'] = 0.1026
df.loc[df.index == datetime.date(2020, 10, 15).isoformat(), 'trac_usd'] = 0.1
df.loc[df.index == datetime.date(2020, 10, 31).isoformat(), 'trac_usd'] = 0.09455

In [131]:
# Fix data errors in coinmarketcap
df['gnt_usd_mcap'] = np.nan
df['spnd_usd_mcap'] = np.nan

In [135]:
df.to_pickle('../3-data/raw/coinapi_price_mcap_volume_data.pkl')

### (5) Obtain data for other tokens to potentially use later

In [314]:
tokens_processed = tokens.copy()

In [315]:
# Find the coins available non US but MAJOR exchanges
relevant_exchanges = ['HUOBI', 'BINANCE', 'FTX', 'OKEX', 'BITFINEX']
rel_nonus_symbols = []
for symbol in symbols:
    if (symbol['exchange_id'] in relevant_exchanges):
        if (symbol['symbol_type'] in ['SPOT']):
            rel_nonus_symbols.append(symbol)

In [316]:
# Find all unique tokens
temp_list = []
for symbol in rel_nonus_symbols:
    temp_list.append(symbol['asset_id_base'])
unique_tokens = list(np.unique(np.array(temp_list)))


In [317]:
# Remove tokens that are derivatives of other tokens
for string in token_names_to_remove:
    unique_tokens = [i for i in unique_tokens if not (string in i)]



In [318]:
# Remove stable coins    
unique_tokens = list(unique_tokens)
currencies_to_remove = ['EUR', 'AUD', 'GBP', 'CAD', 
                        'MTL', 'ERN']
for c in currencies_to_remove:
    unique_tokens.remove(c)


In [319]:
coinapi_df = pd.DataFrame(data = unique_tokens)
coinapi_df = coinapi_df.rename(columns = {0: 'symbol'})

# Merge together messari coins with coin api tokens
token_df = messari_df.merge(coinapi_df,
                            how='outer',
                            on='symbol',
                            validate='one_to_one',
                            indicator=True)

# Remove tokens manually
symbols_dont_map_to_token = ['ACB', 'AFCWIN-SB-2021', 'AMC', 'AMD', 'ANIO',
                             'ARN', 'ATO', 'AXP', 'B21X', 'BARY', 'BB',
                             'BCHA', 'BCHN', 'BCHSV', 'BKCAT', 
                             'BTMX', 'BT1', 'BT2', 'CFD', 'CGLD', 'CSTBCHABC',
                             'CSTBCHN', 'DADI', 'ENDOR',
                             'ETH2', 'ETH2.S', 'GALAX', 'HAT', 'HAV',
                             'HKN', 'HMCN', 'HOTNOW', 'HOTT', 'HSR',
                             'ICN', 'INS', 'OPQ', 'MCO2', 'MODEFI', 
                             'NFCWIN-SB-2021', 'OK06ETT', 'PAX', 'PRL', 'REPV2',
                             'RHOC', 'RPX', 'SHL', 'SNTR', 
                             'TERRAUST', 'TESTA', 'TESTBTC', 'TESTC', 'TESTD',
                             'TIO', 'TRUMPGO', 'TRUMPLOSE', 'TRUMPSTAY',
                             'TRUMPWIN', 'VEN', 'VRAB',
                             'XRB', 'YFDAI', 'YOP', 'ZEL']
symbols_with_token_below_mcap_1m_ever = ['AAA', 'ACOIN', 'BOS', 'CEUR', 'COFI', 'DACC', 'DCC',
                                         'FORESTPLUS', 'GLA', 'J8T', 'LNCHX', 'MUSH', 'ONOT',
                                         'PARETO', 'PAZZI', 'PBL', 'PMGT', 'PURA', 'SNOV',
                                         'SPF', 'STQ', 'TRAXIA', 'UKG', 'VNX', 'VSX', 
                                         'WAX', 'WGP', 'XLR', 'ZINC']
stablecoin_symbols_to_drop = ['CUSD', 'MUSD', 'NUSD', 'DAI', 'CNH']
wrapped_tokens_to_drop = ['AAPL', 'ABNB', 'AMZN', 'ARKK', 'BABA', 'BTC1S', 
                          'BTCST', 'EBTC', 'EFIL', 'RBTC', 'TBTC', 'TSLA',
                          'UNI2L', 'UNI2S', 'WBTC']
token_df = token_df[~(token_df.symbol.isin(symbols_dont_map_to_token))]
token_df = token_df[~(token_df.symbol.isin(symbols_with_token_below_mcap_1m_ever))]
token_df = token_df[~(token_df.symbol.isin(stablecoin_symbols_to_drop))]
token_df = token_df[~(token_df.symbol.isin(wrapped_tokens_to_drop))]

# DROP COINS IN MESSARI BUT NOT PURCHASABLE
token_df = token_df[token_df._merge != 'left_only']
token_df = token_df.drop('_merge', axis=1)


In [320]:
# Drop tokens that i already have data on
token_df = token_df[~(token_df.symbol.isin(tokens_processed))]
token_df = token_df.reset_index(drop=True)

In [321]:
# Build list of tokens to pull historical prices for

tokens = token_df.symbol.values
assert(len(np.unique(tokens)) == len(tokens))
tokens = list(np.unique(tokens)) # sort alphabetically

In [323]:
# Drop tokens that I can't find market for
rel_index = tokens.index('SPARTA')
tokens.pop(rel_index)

'SPARTA'

In [324]:
# Figure the relevant market for all tokens
exchanges_in_order_by_historical_length = ['HUOBI', 'BINANCE', 'FTX', 'OKEX', 'BITFINEX']
markets = []
for i in range(0,len(tokens)):
    token = tokens[i]

    # Determine all the relevant markets for the token
    all_markets = dict.fromkeys(relevant_exchanges, [])
    for symbol in symbols:
        if (symbol['exchange_id'] in relevant_exchanges):
            try:
                if symbol['asset_id_base'] == token:
                    all_markets[symbol['exchange_id']] = all_markets[symbol['exchange_id']] + [symbol['symbol_id']]
            except KeyError:
                pass
    
    # Determine USD market using first exchange with one by volume
    market = ''
    market = findRelevantMarket(token, all_markets, '_BTC', exchanges_in_order_by_historical_length)

    # If no USD market, let's look for BTC, then USDT, then ETH
    if not market:
        market = findRelevantMarket(token, all_markets, '_USDT', exchanges_in_order_by_historical_length)
    if not market:
        market = findRelevantMarket(token, all_markets, '_ETH', exchanges_in_order_by_historical_length)
    if not market:
        market = findRelevantMarket(token, all_markets, '_USD', exchanges_in_order_by_historical_length)
    if not market:
        market = findRelevantMarket(token, all_markets, '_USDC', exchanges_in_order_by_historical_length)
    if not market:
        print("Didn't find a market for " + token)
        break

    assert(type(market) is list)
    market = market[0]
    assert(type(market) is str)
    assert(len(market) > 0) # Ensure we at least found some market
    markets.append(market)
    

In [325]:
# Pull the historical data for all tokens
# NOTE: BIG API CALL!!!
ohlcvs_new = []
start_of_2009 = datetime.date(2009, 1, 1).isoformat()
today = datetime.date(2022, 1, 4).isoformat()
for i in range(0,len(markets)):
    if (i % 20) == 0:
        print(i)
    market = markets[i]
    print(market)
    print('\n')
    ohlcv = api.ohlcv_historical_data(market, {'period_id': '1DAY', 
                                               'time_start': start_of_2009,
                                               'time_end': today,
                                               'limit': 5000})

    ohlcvs_new.append(ohlcv)

0
HUOBI_SPOT_18C_BTC


OKEX_SPOT_1ST_BTC


HUOBI_SPOT_AAC_BTC


OKEX_SPOT_ABL_BTC


BITFINEX_SPOT_ABS_ETH


BINANCE_SPOT_ACM_BTC


HUOBI_SPOT_AE_BTC


BITFINEX_SPOT_AID_BTC


HUOBI_SPOT_AIDOC_BTC


BITFINEX_SPOT_AIO_BTC


OKEX_SPOT_AKITA_USDT


BITFINEX_SPOT_ALG_BTC


OKEX_SPOT_ALV_USDT


OKEX_SPOT_AMM_BTC


OKEX_SPOT_ANW_USDT


BINANCE_SPOT_ANY_BTC


FTX_SPOT_APHA_USD


OKEX_SPOT_APIX_BTC


OKEX_SPOT_APM_BTC


HUOBI_SPOT_APPC_BTC


20
HUOBI_SPOT_ARDR_BTC


BINANCE_SPOT_ARK_BTC


BINANCE_SPOT_ASR_BTC


HUOBI_SPOT_AST_BTC


OKEX_SPOT_ATL_BTC


FTX_SPOT_ATLAS_USD


BINANCE_SPOT_ATM_BTC


HUOBI_SPOT_ATP_BTC


BITFINEX_SPOT_AUC_BTC


BINANCE_SPOT_AUTO_BTC


OKEX_SPOT_AVT_BTC


OKEX_SPOT_BABYDOGE_USDT


FTX_PERP_BAO_USD


BITFINEX_SPOT_BBN_ETH


BITFINEX_SPOT_BCC_BTC


BITFINEX_SPOT_BCI_BTC


BINANCE_SPOT_BCN_BTC


HUOBI_SPOT_BCV_BTC


HUOBI_SPOT_BCX_BTC


BINANCE_SPOT_BEAM_BTC


40
OKEX_SPOT_BEC_BTC


BINANCE_SPOT_BEL_BTC


BITFINEX_SPOT_BEST_USD


BINANCE_SPOT_BETA_BTC


OKEX_SPOT_BETH_US

OKEX_SPOT_PST_BTC


HUOBI_SPOT_PVT_BTC


FTX_SPOT_PYPL_USD


HUOBI_SPOT_QASH_BTC


BITFINEX_SPOT_QSH_BTC


BITFINEX_SPOT_QTF_BTC


HUOBI_SPOT_QUN_BTC


OKEX_SPOT_QVT_BTC


OKEX_SPOT_RACA_USDT


BINANCE_SPOT_RAMP_BTC


380
HUOBI_SPOT_RCCC_BTC


HUOBI_SPOT_RCN_BTC


OKEX_SPOT_RCT_BTC


OKEX_SPOT_READ_BTC


FTX_SPOT_REAL_USD


OKEX_SPOT_REF_BTC


HUOBI_SPOT_RENBTC_BTC


OKEX_SPOT_RFR_BTC


HUOBI_SPOT_RING_BTC


BITFINEX_SPOT_RINGX_USD


OKEX_SPOT_RIO_USDT


OKEX_SPOT_RNT_BTC


FTX_SPOT_ROOK_USDT


BITFINEX_SPOT_RRB_USDT


BITFINEX_SPOT_RRT_BTC


HUOBI_SPOT_RTE_BTC


HUOBI_SPOT_RUFF_BTC


HUOBI_SPOT_SALT_BTC


OKEX_SPOT_SAMO_USDT


OKEX_SPOT_SAN_BTC


400
HUOBI_SPOT_SBTC_BTC


BITFINEX_SPOT_SCR_ETH


BINANCE_SPOT_SCRT_BTC


OKEX_SPOT_SDA_BTC


FTX_SPOT_SECO_USD


BITFINEX_SPOT_SEE_BTC


HUOBI_SPOT_SEELE_BTC


BITFINEX_SPOT_SEN_BTC


OKEX_SPOT_SFG_USDT


FTX_SPOT_SGD_USD


HUOBI_SPOT_SHE_BTC


OKEX_SPOT_SHOW_BTC


HUOBI_SPOT_SKM_BTC


BINANCE_SPOT_SKY_BTC


FTX_SPOT_SLND_USD


FTX_SPOT_SLRS

In [363]:
# Tokens to remove
tokens_to_remove = ['ATP', 'BHD', 'BHP', 'BHT',
                    'BOT', 'CNNS', 'COVA', 'CRE',
                    'CRU', 'CVNT', 'WGRT']
for remove_token in tokens_to_remove:
    rel_index = tokens.index(remove_token)
    tokens.pop(rel_index)
    markets.pop(rel_index)
    ohlcvs_new.pop(rel_index)

In [364]:
# Convert ohlcv objects to data frames
dfs = []
for i in range(0, len(ohlcvs_new)):
    ohlcv = ohlcvs_new[i]
    market = markets[i]
    token = tokens[i]
    print(token)
    if ohlcv != []:
        df = cleanOHLCV(ohlcv, market, token, 
                        btc_df=btc_df, eth_df=eth_df, usdt_df=usdt_df, usdc_df=usdc_df)
        dfs.append(df)

# Merge together all data frames to save
df = dfs[0]
for i in range(1,len(dfs)):
    df = df.merge(dfs[i],
                  how='outer',
                  on='date',
                  validate='one_to_one')
    
df = df.sort_values('date')
df = df.reset_index(drop=True)

18C
1ST
AAC
ABL
ABS
ACM
AE
AID
AIDOC
AIO
AKITA
ALG
ALV
AMM
ANW
ANY
APHA
APIX
APM
APPC
ARDR
ARK
ASR
AST
ATL
ATLAS
ATM
AUC
AUTO
AVT
BABYDOGE
BAO
BBN
BCC
BCI
BCN
BCV
BCX
BEAM
BEC
BEL
BEST
BETA
BETH
BFT
BFX
BGBP
BIFI
BILI
BITO
BITR
BITW
BIX
BKBT
BKRW
BKX
BLT
BLZ
BMI
BNTX
BNX
BOBA
BORING
BOX
BRL
BTCB
BTS
BTSE
BTU
BUT
BVOL
BYND
BZZ
CAI
CBSE
CBT
CDT
CEL
CELR
CELT
CFI
CFX
CGC
CGS
CHAT
CHE
CHESS
CHEX
CIC
CIT
CITY
CLO
CLOAK
CND
CNN
CNTM
COCOS
COIN
COMET
CONV
COPE
COS
COVER
CRON
CSPR
CSX
CTC
CTK
CTXC
CVCOIN
CVP
CVT
CVX
DAC
DAD
DAPP
DAR
DAWN
DEP
DF
DFL
DGD
DGX
DHT
DKA
DKNG
DLT
DMD
DOG
DOSE
DOT2L
DOT2S
DPY
DREP
DRK
DRN
DTH
DTX
DUSK
DVF
EASY
EC
EDEN
EDO
EDU
EFI
EGCC
EKO
EKT
ELC
EM
EMB
ENG
EOSDT
EOSH
ERD
ESS
ETH1S
ETH2X
ETHE
ETM
ETP
EUS
EUT
EVT
EXE
EXO
EZ
FAIR
FB
FEG
FIDA
FIO
FIRO
FIS
FLM
FOA
FODL
FOR
FSN
FTI
FUEL
FUN
GAL
GAMECOM
GBTC
GDX
GDXJ
GEN
GENE
GET
GLD
GLXY
GM
GME
GNX
GOF
GOOGL
GRS
GSC
GSD
GT
GTO
GTX
GVE
GXC
GXS
HB10
HCC
HDAO
HEZ
HGET
HIGH
HIT
HIVE
HMC
HMT
HOLY
HOOD
HOPL
HOT
HPT
HUM
HXRO
HYC


In [365]:
len(tokens)

527

In [366]:
len(markets)

527

In [368]:
len(ohlcvs_new)

527

In [383]:
df.to_pickle('../3-data/raw/coinapi_price_volume_data_nonus_exchanges.pkl')